In [1]:
from mitiq import zne, benchmarks, MeasurementResult
from qiskit import *
import networkx as nx
from qiskit_aer import AerSimulator
from collections import defaultdict
from qiskit.circuit import Gate
import numpy as np



In [2]:
def execute(circuit, backend=AerSimulator()):
    depth = circuit.depth()
    ops_dict = circuit.count_ops()
    counts = defaultdict(int)

    for inst in circuit.data:
        if isinstance(inst.operation, Gate):
            counts[len(inst.qubits)] += 1

    print('1 and 2 qubit operator counts: ', {k: v for k, v in counts.items()})

    print('circuit depth: ', depth)
    print('Individual operator counts: ', {k: v for k, v in ops_dict.items()})

    circuit.measure_all()
    transpiled = transpile(circuit, backend=backend, optimization_level=0)
    transpiled_depth = transpiled.depth()
    print('Transpiled circuit depth: ', transpiled_depth)


    result = backend.run(circuit, shots=10, optimization_level=0).result()
    counts = result.get_counts(circuit)
    counts = { k.replace(" ",""):v for k, v in counts.items()}
    measurements = MeasurementResult.from_counts(counts)

    print(measurements, '\n')
    return 0

In [3]:
n_qubits = 2
connectivity_graph = nx.complete_graph(n_qubits)


def get_circuit(depth, seed):
    circuit, correct_bitstring = benchmarks.generate_mirror_circuit(
        nlayers=depth,
        two_qubit_gate_prob=1.0,
        connectivity_graph=connectivity_graph,
        two_qubit_gate_name="CNOT",
        seed=seed,
        return_type="qiskit",
    )        
    return circuit, correct_bitstring 

circ, bitstring = get_circuit(4,10)
print(circ)
print("\nShould sample:", bitstring)

     ┌─────────┐   ┌────┐   ┌───┐     ┌───┐     ┌───┐     ┌───┐     ┌───┐     »
q_0: ┤ Ry(π/2) ├───┤ √X ├───┤ I ├──■──┤ Y ├──■──┤ Y ├──■──┤ I ├──■──┤ X ├──■──»
     └──┬───┬──┘┌──┴────┴──┐├───┤┌─┴─┐├───┤┌─┴─┐├───┤┌─┴─┐├───┤┌─┴─┐├───┤┌─┴─┐»
q_1: ───┤ X ├───┤ Ry(-π/2) ├┤ X ├┤ X ├┤ X ├┤ X ├┤ X ├┤ X ├┤ X ├┤ X ├┤ I ├┤ X ├»
        └───┘   └──────────┘└───┘└───┘└───┘└───┘└───┘└───┘└───┘└───┘└───┘└───┘»
«     ┌───┐     ┌───┐     ┌───┐     ┌───┐  ┌──────┐ ┌──────────┐
«q_0: ┤ I ├──■──┤ Z ├──■──┤ Z ├──■──┤ X ├──┤ √Xdg ├─┤ Ry(-π/2) ├
«     ├───┤┌─┴─┐├───┤┌─┴─┐├───┤┌─┴─┐├───┤┌─┴──────┴┐└──┬───┬───┘
«q_1: ┤ Z ├┤ X ├┤ X ├┤ X ├┤ X ├┤ X ├┤ Y ├┤ Ry(π/2) ├───┤ X ├────
«     └───┘└───┘└───┘└───┘└───┘└───┘└───┘└─────────┘   └───┘    

Should sample: [1, 0]


In [4]:
circ2 = QuantumCircuit(2)
circ2.cx(0,1)
circ2.h(1)
circ2.rx(np.pi/2, 0)
circ2.measure_all()
circ2.draw()

┌─────────┐ ░ ┌─┐   
   q_0: ──■──┤ Rx(π/2) ├─░─┤M├───
        ┌─┴─┐└──┬───┬──┘ ░ └╥┘┌─┐
   q_1: ┤ X ├───┤ H ├────░──╫─┤M├
        └───┘   └───┘    ░  ║ └╥┘
meas: 2/════════════════════╩══╩═
                            0  1

In [5]:
print(circ.data)

[CircuitInstruction(operation=Instruction(name='ry', num_qubits=1, num_clbits=0, params=[1.5707963267948966]), qubits=(Qubit(QuantumRegister(2, 'q'), 0),), clbits=()), CircuitInstruction(operation=Instruction(name='sx', num_qubits=1, num_clbits=0, params=[]), qubits=(Qubit(QuantumRegister(2, 'q'), 0),), clbits=()), CircuitInstruction(operation=Instruction(name='id', num_qubits=1, num_clbits=0, params=[]), qubits=(Qubit(QuantumRegister(2, 'q'), 0),), clbits=()), CircuitInstruction(operation=Instruction(name='x', num_qubits=1, num_clbits=0, params=[]), qubits=(Qubit(QuantumRegister(2, 'q'), 1),), clbits=()), CircuitInstruction(operation=Instruction(name='ry', num_qubits=1, num_clbits=0, params=[-1.5707963267948966]), qubits=(Qubit(QuantumRegister(2, 'q'), 1),), clbits=()), CircuitInstruction(operation=Instruction(name='x', num_qubits=1, num_clbits=0, params=[]), qubits=(Qubit(QuantumRegister(2, 'q'), 1),), clbits=()), CircuitInstruction(operation=Instruction(name='cx', num_qubits=2, num_

In [6]:
zne_value = zne.execute_with_zne(circ, executor=execute)

1 and 2 qubit operator counts:  {1: 26, 2: 8}
circuit depth:  21
Individual operator counts:  {'x': 10, 'cx': 8, 'ry': 4, 'id': 4, 'y': 3, 'z': 3, 'sx': 1, 'sxdg': 1}
Transpiled circuit depth:  22
MeasurementResult: {'nqubits': 2, 'qubit_indices': (0, 1), 'shots': 10, 'counts': {'01': 10}} 

1 and 2 qubit operator counts:  {1: 48, 2: 20}
circuit depth:  49
Individual operator counts:  {'cx': 20, 'x': 16, 'z': 9, 'id': 8, 'y': 7, 'ry': 6, 'sx': 1, 'sxdg': 1}
Transpiled circuit depth:  50
MeasurementResult: {'nqubits': 2, 'qubit_indices': (0, 1), 'shots': 10, 'counts': {'01': 10}} 

1 and 2 qubit operator counts:  {1: 78, 2: 24}
circuit depth:  63
Individual operator counts:  {'x': 30, 'cx': 24, 'ry': 12, 'id': 12, 'y': 9, 'z': 9, 'sx': 3, 'sxdg': 3}
Transpiled circuit depth:  64
MeasurementResult: {'nqubits': 2, 'qubit_indices': (0, 1), 'shots': 10, 'counts': {'01': 10}} 

